In [2]:
if Sys.iswindows()
    username = "Alex"
    pathtorepo = "C:\\Users\\" *username *  "\\Desktop\\"
    using Pkg
    Pkg.activate(pathtorepo * "dynamical-systems\\env\\integrate\\")
else
    username = "sergey"
    pathtorepo = "/home/" *username *"/work/repo/dynamical-systems"
    using Pkg
    Pkg.activate(pathtorepo * "/env/integrate/")
    include(pathtorepo * "/system.jl");
    include("/home/sergey/work/repo/dynamical-systems/FHN/jacobian.jl");
end

  Activating project at `~/work/repo/dynamical-systems/env/integrate`


two_FHN_jac (generic function with 1 method)

In [4]:
using DifferentialEquations, DynamicalSystems, StaticArrays, CairoMakie
using LinearAlgebra, ForwardDiff

In [33]:
params = two_coupled_fhn_get_params();
params[7] = 0.09
params[8] = 75.76
u0 = SVector(-1.0092493342265774, -0.6369086486640424, -1.010765472352876, -0.6365947579973191, 1.6183689231427258);
t_integrate = 1500.0;
tspan = (0.0, t_integrate);
integ_set = (alg = RK4(), adaptive = false, dt = 0.001)

(alg = RK4(; stage_limiter! = trivial_limiter!, step_limiter! = trivial_limiter!, thread = static(false),), adaptive = false, dt = 0.001)

In [35]:
ds = CoupledODEs(two_coupled_fhn, u0, params, diffeq = integ_set)

5-dimensional CoupledODEs
 deterministic: true
 discrete time: false
 in-place:      false
 dynamic rule:  two_coupled_fhn
 ODE solver:    RK4
 ODE kwargs:    (adaptive = false, dt = 0.001)
 parameters:    [0.01, -1.01, 160.0, 0.1, 50.0, 50.0, 0.09, 75.76]
 time:          0.0
 state:         [-1.0092493342265774, -0.6369086486640424, -1.010765472352876, -0.6365947579973191, 1.6183689231427258]


In [36]:
Jacobi(u0) = ForwardDiff.jacobian((x) -> two_coupled_fhn(x, params, 0.0), u0)
Jacobi(u0)

5×5 SMatrix{5, 5, Float64, 25} with indices SOneTo(5)×SOneTo(5):
 -19853.3  -100.0     19830.0    34.0211  -37.178
      1.0     0.0         0.0     0.0       0.0
  19830.1    33.811  -19853.6  -100.0      37.178
      0.0     0.0         1.0     0.0       0.0
      1.0     0.0        -1.0     0.0       0.0

In [39]:
det(Jacobi(u0))

0.0

In [38]:
two_FHN_jac(u0, params, 0.0)

5×5 SMatrix{5, 5, Float64, 25} with indices SOneTo(5)×SOneTo(5):
 -12271.6  -100.0     12291.2   -34.0211  -11.4863
      1.0     0.0         0.0     0.0       0.0
  12291.1   -33.811  -12271.9  -100.0      11.4863
      0.0     0.0         1.0     0.0       0.0
      1.0     0.0        -1.0     0.0       0.0

In [40]:
det(two_FHN_jac(u0, params, 0.0))

0.0

In [10]:
tands = TangentDynamicalSystem(ds; J = two_FHN_jac)

5-dimensional TangentDynamicalSystem
 deterministic:     true
 discrete time:     false
 in-place:          false
 dynamic rule:      two_coupled_fhn
 jacobian:          two_FHN_jac
 deviation vectors: 5
 parameters:        [0.01, -1.01, 160.0, 0.1, 50.0, 50.0, 0.09, 0.0025]
 time:              0.0
 state:             [-1.0100851158481972, -0.6365889912684914, -1.0099261711695993, -0.6369147033834776, 1.619008525924632]


In [11]:
lyapunovspectrum(tands, 20000)

5-element Vector{Float64}:
   9.484204700069785
   9.431460336553815
  -0.0005092980885129175
 -Inf
 -Inf

In [12]:
lyapunovspectrum(ds, 20000)

5-element Vector{Float64}:
   0.024151559157269357
  -2.66338528122942e-5
  -0.00035290586711944426
 -Inf
 -Inf